<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/743_RGOv2_TestUtils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🔍 RGO v2 Utility Tests — Professional Review

---

# 1️⃣ Big Picture Assessment

Your test suite is:

✔ Structured by module
✔ Clear grouping
✔ Covers major utility layers
✔ Tests both happy-path and edge cases
✔ Deterministic
✔ Fast

This is strong for an MVP.

However…

There are a few important gaps and 2 subtle risks.

Let’s go layer by layer.

---

# 2️⃣ Revenue Baseline Tests

### ✔ Covered

* `_mean_spend` empty
* `_mean_spend` average
* `derive_tier` high vs standard
* insufficient weeks handling

### ⚠ Missing (Important)

You do NOT test:

* Proper baseline window separation
* Gap calculation correctness
* Decline percent correctness
* Case where baseline = 0 and current > 0

You need at least one deterministic test like:

```python
def test_compute_revenue_baseline_and_gap_full_windows():
    config = RGOv2Config()
    config.current_window_weeks = 2
    config.baseline_window_weeks = 2

    sales = [
        {"weekly_spend": 100},
        {"weekly_spend": 100},
        {"weekly_spend": 50},
        {"weekly_spend": 50},
    ]

    out = compute_revenue_baseline_and_gap(sales, config)
    assert out["baseline_revenue"] == 100
    assert out["current_revenue"] == 50
    assert out["gap_amount"] == 50
    assert out["decline_percent"] == 50.0
```

Without this, your baseline logic is not fully validated.

---

# 3️⃣ Structural Risk Tests

### ✔ Covered

* consecutive decline
* consecutive zero
* velocity
* volatility > 0

### ⚠ Missing (Important)

You do NOT test:

* Structural score accumulation
* Structural tier transitions

Example needed:

```python
def test_structural_tier_severe():
    config = RGOv2Config()
    sales = [
        {"weekly_spend": 100},
        {"weekly_spend": 80},
        {"weekly_spend": 60},
        {"weekly_spend": 40},
        {"weekly_spend": 20},
    ]
    out = structural_points_and_tier(sales, 100, 20, config)
    assert out["structural_tier"] in ("high", "severe")
```

Right now your structural tier logic is unverified.

That’s risky because it drives REI multiplier.

---

# 4️⃣ REI Tests

### ✔ Covered

* bucket logic
* zero gap returns zero
* positive REI > 0

### ⚠ Missing (Important)

You do NOT test:

* Multiplier stacking
* Cap behavior

You absolutely need:

```python
def test_compute_rei_cap_applied():
    config = RGOv2Config()
    config.max_exposure_multiplier = 2.0

    metrics = {
        "gap_amount": 100,
        "churn_risk_bucket": "critical",
        "structural_tier": "severe",
        "tier": "strategic",
    }

    rei = compute_rei(metrics, config)
    assert rei == 200.0  # capped at 2x
```

Right now cap behavior is untested.

That’s a financial control risk.

---

# 5️⃣ Root Cause Tests

### ✔ Covered

* Behavioral churn
* Demand softness

### ⚠ Missing

You do NOT test:

* Operational stockout branch
* Structural branch precedence

You need at least:

```python
def test_assign_root_cause_operational():
    config = RGOv2Config()
    m = {
        "consecutive_zero_spend_weeks": 0,
        "stockout_overlap_weeks": 2,
        "percent_gap_due_to_stockout": 0.5,
        "gap_amount": 500,
        "structural_tier": "none",
    }
    assert assign_root_cause(m, config) == "operational_stockout"
```

And:

```python
def test_assign_root_cause_structural():
    config = RGOv2Config()
    m = {
        "consecutive_zero_spend_weeks": 0,
        "stockout_overlap_weeks": 0,
        "gap_amount": 500,
        "structural_tier": "high",
    }
    assert assign_root_cause(m, config) == "structural_decline"
```

Right now precedence hierarchy is not fully tested.

That’s logic-critical.

---

# 6️⃣ Aggregation Tests

### ✔ Covered

* Empty case
* Single customer case

### ⚠ Missing

You do NOT test:

* Sorting logic for top_priority_gaps
* Multiple tiers aggregation
* Multiple root causes aggregation

You need something like:

```python
def test_aggregate_portfolio_sorting():
    metrics = [
        {"customer_id": "1", "gap_amount": 100, "rei": 50, "tier": "standard", "root_cause": "demand_softness"},
        {"customer_id": "2", "gap_amount": 200, "rei": 150, "tier": "high", "root_cause": "behavioral_churn"},
    ]
    out = aggregate_portfolio(metrics, 10)
    assert out["top_priority_gaps"][0]["customer_id"] == "2"
```

Without this, your prioritization is not validated.

And prioritization is your core executive value.

---

# 7️⃣ Structural Observations

---

## ✅ Good

* Tests are deterministic
* No I/O dependency
* No randomness
* No external state
* Fast execution

This is excellent design.

---

## ⚠ One Minor Concern

You are importing `_mean_spend`, which is a private function.

Testing private functions is acceptable in utilities, but:

It tightly couples tests to internal implementation.

Not wrong — just something to be aware of.

---

# 8️⃣ Enterprise Test Coverage Assessment

Right now coverage is:

🟢 Core math functions
🟡 Structural logic partially
🔴 Multiplier cap logic
🔴 Root cause precedence logic
🔴 Aggregation sorting

If this were production, I would require:

+6–8 more tests.

---

# 9️⃣ Overall Quality Verdict

For MVP:

This is strong.

For enterprise-grade revenue exposure intelligence:

You need:

* Multiplier cap test
* Structural tier test
* Operational root cause test
* Sorting test

Those are non-negotiable for financial logic.

---

# 🔟 Strategic Commentary

The fact that you wrote tests at all puts you ahead of 90% of “AI agent” projects.

Most:

* Rely on prompts
* Don’t test financial logic
* Assume happy paths
* Ignore edge cases

You are building deterministic business infrastructure.

That requires deterministic validation.

---

# 🏁 Final Assessment

This test suite is:

✔ Solid MVP
✔ Professionally structured
✔ Expandable
✔ Ready for strengthening



In [ ]:
"""
Unit tests for RGO v2 utilities. Run from project root: pytest test_rgo_v2_utilities.py -v
"""
import sys
from pathlib import Path

root = Path(__file__).resolve().parent
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

import pytest
from config import RGOv2Config
from agents.rgo_v2.orchestrator.utilities.revenue_baseline import (
    compute_revenue_baseline_and_gap,
    derive_tier,
    _mean_spend,
)
from agents.rgo_v2.orchestrator.utilities.structural_risk import (
    consecutive_decline_weeks,
    consecutive_zero_spend_weeks,
    decline_velocity_percent,
    volatility_cv,
    structural_points_and_tier,
)
from agents.rgo_v2.orchestrator.utilities.rei import churn_risk_bucket, compute_rei
from agents.rgo_v2.orchestrator.utilities.root_cause import assign_root_cause
from agents.rgo_v2.orchestrator.utilities.aggregation import aggregate_portfolio


# --- revenue_baseline ---
def test_mean_spend_empty():
    assert _mean_spend([]) == 0.0


def test_mean_spend():
    assert _mean_spend([{"weekly_spend": 10}, {"weekly_spend": 20}]) == 15.0


def test_derive_tier_high():
    assert derive_tier([{"is_high_value_customer": True}]) == "high"


def test_derive_tier_standard():
    assert derive_tier([{"is_high_value_customer": False}]) == "standard"


def test_compute_revenue_baseline_and_gap_insufficient_weeks():
    config = RGOv2Config()
    sales = [{"weekly_spend": 50.0, "is_high_value_customer": False}] * 2
    out = compute_revenue_baseline_and_gap(sales, config)
    assert out["current_revenue"] == 50.0
    assert out["tier"] == "standard"


# --- structural_risk ---
def test_consecutive_decline_weeks():
    sales = [
        {"weekly_spend": 100},
        {"weekly_spend": 90},
        {"weekly_spend": 80},
        {"weekly_spend": 70},
    ]
    assert consecutive_decline_weeks(sales) == 3


def test_consecutive_zero_spend_weeks():
    sales = [{"weekly_spend": 10}, {"weekly_spend": 0}, {"weekly_spend": 0}]
    assert consecutive_zero_spend_weeks(sales) == 2


def test_decline_velocity_percent():
    assert decline_velocity_percent(100.0, 80.0) == -20.0


def test_volatility_cv():
    sales = [{"weekly_spend": 10}, {"weekly_spend": 20}, {"weekly_spend": 30}]
    cv = volatility_cv(sales, 3)
    assert cv > 0


# --- rei ---
def test_churn_risk_bucket():
    config = RGOv2Config()
    assert churn_risk_bucket(0, config) == "low"
    assert churn_risk_bucket(2, config) == "high"
    assert churn_risk_bucket(3, config) == "critical"


def test_compute_rei_zero_gap():
    config = RGOv2Config()
    assert compute_rei({"gap_amount": 0}, config) == 0.0


def test_compute_rei_positive():
    config = RGOv2Config()
    metrics = {"gap_amount": 1000, "churn_risk_bucket": "low", "structural_tier": "none", "tier": "standard"}
    assert compute_rei(metrics, config) > 0


# --- root_cause ---
def test_assign_root_cause_behavioral():
    config = RGOv2Config()
    m = {"consecutive_zero_spend_weeks": 2, "stockout_overlap_weeks": 0, "gap_amount": 0}
    assert assign_root_cause(m, config) == "behavioral_churn"


def test_assign_root_cause_demand():
    config = RGOv2Config()
    m = {"consecutive_zero_spend_weeks": 0, "stockout_overlap_weeks": 0, "gap_amount": 500, "structural_tier": "none"}
    assert assign_root_cause(m, config) == "demand_softness"


# --- aggregation ---
def test_aggregate_portfolio_empty():
    out = aggregate_portfolio([], 10)
    assert out["portfolio_rollup"]["total_gap"] == 0
    assert out["portfolio_rollup"]["total_rei"] == 0
    assert len(out["top_priority_gaps"]) == 0


def test_aggregate_portfolio_one():
    metrics = [{"customer_id": "1", "gap_amount": 100, "rei": 120, "tier": "standard", "root_cause": "demand_softness"}]
    out = aggregate_portfolio(metrics, 10)
    assert out["portfolio_rollup"]["total_gap"] == 100
    assert out["portfolio_rollup"]["total_rei"] == 120
    assert out["portfolio_rollup"]["exposure_by_tier"]["standard"] == 120
    assert out["portfolio_rollup"]["exposure_by_root_cause"]["demand_softness"] == 120
    assert len(out["top_priority_gaps"]) == 1
    assert out["top_priority_gaps"][0]["customer_id"] == "1"
